# Import Data

In [1]:
from os import listdir
from os.path import isfile, join
data_path = './review40/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

In [2]:
onlyfiles[:5]

['000 Myeongdong_Shopping_Street-Seoul.csv',
 '001 Myeong_dong_Cathedral-Seoul.csv',
 '002 Chuncheon_Myeongdong_Street-Chuncheon_Gangwon_do.csv',
 '003 Gwangandaegyo_Bridge-Busan.csv',
 '004 Nurimaru_APAC_House-Busan.csv']

In [3]:
import pandas as pd

In [4]:
documents = pd.DataFrame(columns=['attraction', 'review'])

for idx, file_name in enumerate(onlyfiles):
    data = pd.read_csv(data_path + onlyfiles[idx], error_bad_lines=False)
    data2 = data.apply(lambda x: x[0] + '. ' + x[1], axis=1)
    data3 = data2 + ' '
    data4 = data3.sum()
    documents.loc[idx] = [onlyfiles[idx].split('.')[0], data4]

documents.attraction = documents.attraction.apply(lambda x: x[4:])

# Mapping 

{168개의 관광지} $\rightarrow$ {(40 - 1(스키장))개의 관광지}

우리가 추천해줄 수 있는 관광지는 534개가 있지만 설문조사에서 만족방문지에 작성된 top40의 범주에 맞춰준다.

참고로 category 0은 40개의 범주에 속하지 못한 관광지를 의미한다.

한국 관광조사에서 만족 관광지를 기록할 때 구체적으로 조사하지 않아서 이 과정을 하는 것인데 리뷰를 묶을 때 그냥 쌩으로 더 하도록 하겠다.

예시

> A 관광지 B 관광지 C 관광지를 "*N*" 번 범주로 묶는 과정을 한다.
>
> "*N*" 이라는 관광지에는 정말 다양한 관광지들이 속해 있는데 모든 관광지를 같은 가중치로 준다고 생각하면
>
> 리뷰의 텍스트를 길이를 맞춰주고 묶어야지 A관광지 B관광지 C관광지의 특성을 골고루 반영될테지만 
> 
> A지역은 자주 방문하는 관광지이고 C지역은 거의 가지 않는 편이라면 이는 리뷰의 숫자로 반영될 것이고
> 
> 한국 관광조사에서 "*N*"이라고 적을 때 A지역을 간 비율이 높을 것이므로

In [5]:
mapping = pd.read_csv('./mapping/mapping.csv')

mapping['category'] = mapping.category.fillna(0).astype(int)

documents = pd.merge(documents, mapping, how='left')

documents = documents.reset_index()

documents.columns = ['attr_index', 'attraction', 'review', 'category']

documents.attr_index = documents.attr_index

In [6]:
def custom_merge(merge_df):
    return (merge_df['attraction'].str.cat(sep=' '), merge_df['review'].str.cat(sep=' '), merge_df['attr_index'].to_list())

documents_merge = documents.groupby('category').apply(lambda x: custom_merge(x))

documents_merge = pd.DataFrame({'attractions' : documents_merge.apply(lambda x: x[0]), 'review' : documents_merge.apply(lambda x: x[1]),
                               'attr_index' : documents_merge.apply(lambda x: x[2])})

documents_merge = documents_merge.iloc[1:, :] # remove category 0 

documents_merge = documents_merge.reset_index()

# Data Pre-processing

* Tokenization
> lower, punctuation 제거, 글자 길이 3개 미만 단어 제거, stopwords제거, 

* stopwords
> 제거

* 글자 길이 3개 미만 단어
> 제거

In [7]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MASTER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
stemmer = PorterStemmer()

In [9]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

어떻게 전처리가 되는 지 보자!

In [10]:
WTS = 0 # want to see
REVIEW_COLUMNS = 2 # review column is 1st

doc_sample = documents_merge.loc[WTS].values[REVIEW_COLUMNS]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words[:10])
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample)[:10])

original document: 
["It's", 'cosmetics,', 'cosmetics', 'and', 'more', 'cosmetics.', '.', 'Interesting', 'place', 'if']


 tokenized and lemmatized document: 
['cosmet', 'cosmet', 'cosmet', 'interest', 'place', 'look', 'cosmet', 'street', 'food', 'restaur']


In [11]:
processed_docs = documents_merge['review'].map(preprocess)
processed_docs

0     [cosmet, cosmet, cosmet, interest, place, look...
1     [wholesal, shop, coupl, stop, myeongdong, stay...
2     [tourist, religi, devot, beauti, templ, defini...
3     [typic, seoul, live, sinchon, area, foreign, t...
4     [even, great, neighborhood, even, feel, like, ...
5     [prettyyyyyy, friend, fell, love, sight, night...
6     [price, expens, good, shop, price, reason, tim...
7     [basebal, game, korea, experi, trip, south, ko...
8     [drop, visit, market, whilst, citi, rout, mass...
9     [good, earli, amaz, experi, korea, probabl, fa...
10    [nice, visit, fish, market, load, stall, resta...
11    [aquarium, great, age, right, time, shark, fee...
12    [huge, park, lot, space, park, huge, tire, cro...
13    [time, friend, good, time, enjoy, rid, wait, h...
14    [littl, differ, templ, aren, usual, locat, oce...
15    [arti, villag, histori, love, littl, villag, t...
16    [king, joseon, admir, shin, domin, squar, larg...
17    [amaz, boutiqu, shop, interest, boutiqu, i

In [12]:
dictionary = gensim.corpora.Dictionary(processed_docs)

# dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 aajuma
1 aant
2 abalon
3 abil
4 abit
5 abl
6 abound
7 abour
8 abroad
9 abruptli
10 absolut


In [13]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs] # word frequency for each attraction
len(bow_corpus)

39

# LDA

## LDA Train

perplexity에 기반하지 않고 Toipc 개수를 4개로 제한한 이유

1. 주제는 적어도 어렵지만 많아져도 해석하기 어려워진다.

2. 고차원 심플렉스를 분석하여 추천을 하는 것이 찾을 수록 어렵다.

3. 심플렉스를 온전히 시각화하기 위해서

In [14]:
NUM_TOPICS = 4 #4개의 토픽, k=4
ldamodel = gensim.models.ldamodel.LdaModel(bow_corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15, random_state=50)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.017*"place" + 0.016*"park" + 0.012*"visit" + 0.012*"beach"')
(1, '0.042*"shop" + 0.024*"food" + 0.024*"street" + 0.023*"place"')
(2, '0.019*"walk" + 0.017*"place" + 0.016*"visit" + 0.013*"tour"')
(3, '0.020*"view" + 0.015*"place" + 0.012*"walk" + 0.011*"jeju"')


각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도를 보여줍니다. 

또한 맨 앞에 있는 토픽 번호는 0부터 시작하므로 총 4개의 토픽은 0부터 3까지의 번호가 할당되어져 있습니다.

passes는 알고리즘의 동작 횟수를 말하는데, 알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 충분히 적당한 횟수를 정해주면 됩니다. 

여기서는 총 15회를 수행하였습니다.

여기서는 num_words=4로 총 4개의 단어만 출력하도록 하였습니다.

## LDA Visualization

In [15]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, bow_corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\MASTER\Anaconda3\envs\datascience\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


$$r(w, k | \lambda)=\lambda \log \left(\phi_{k w}\right)+(1-\lambda) \log \left(\frac{\phi_{k w}}{p_{w}}\right)$$

lambda가 1이면 토픽이 해당 word를 뱉는 확률로 rank를 세우고

lambda가 0이면 토픽이 해당 word를 뱉는 확률을 단어가 등장하는 빈도로 조정한 것으로 rank를 세운다.

특정 반복되는 단어가 많을 땐 문서에 대한 토픽을 해석하는 게 어려우니 0에 가깝게 두는 게 용이하다.

$\lambda=0.1$로 둔 다음에 해석을 하면

1번 (도심 - 조용한)
> nami, bike, aquarium, everland, beach, rid, haeunda, hangang, incheon, franc, yeouido, basebal, blossom, cherri, cycl, gapyeong, petit, coex, arex, songdo, safari, sonata, ferri, island, bicycl, roller, layov, cruis, firework, coaster

2번 (도시 - 번화가의)
> shop, street, cosmet, myeongdong, brand, cathedr, food, jeonju, product, cloth, church, fashion, store, skincar, skin, mask, bargain, cathol, namdaemun, accessori, hanok, gamcheon, shopper, makeup, itaewon, lobster, trendi, sampl, heaven, mass

3번 (역사 - 조용한)
> stream, palac, templ, north, fortress, suwon, tunne, lantern, gyeongbokgung, gwanghwamun, gyeongju, shrine, hwaseong, cheonggyecheon, buddha, border, sejong, seoraksan, guard, bulguksa, infiltr, dora, ceremoni, dorasan, soldier, squar, wall, king, histori 

4번 (자연의 자연 - 조용한)
> jeju, seafood, hike, sunris, climb, trail, seongsan, crater, rock, hallasan, peak, diver, ilchulbong, summit, yeongsi, format, volcan, seongpanak, eorimok, mysteri, volcano, dragon, jagalchi, seogwipo, gwaneumsa, yongdusan, lava, dongmun, cliff, neutral

In [16]:
pyLDAvis.save_html(vis, 'lda_viz.html')
pyLDAvis.save_json(vis, 'lda_viz.json')

## LDA topic distribution for each documents

In [15]:
def make_topictable_per_doc(ldamodel, corpus, indices):
    topic_table = pd.DataFrame()
    for i, topic_list in zip(indices, (ldamodel[corpus][idx] for idx in indices)):
        doc = sorted(topic_list, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_list = pd.Series(topic_list).apply(lambda x: (x[0]+1, x[1])).to_list() # 토픽 0 1 2 3==> 1 2 3 4
                topic_table = topic_table.append(pd.Series([int(i), int(topic_num) + 1, round(prop_topic,4), topic_list]), ignore_index=True)
                # int(topic_num) + 1 to match the plotted topic indexing
            else:
                break
    return(topic_table)

In [16]:
topictable = make_topictable_per_doc(ldamodel, bow_corpus, processed_docs.index)
topictable.columns = ['category', 'highest_topic', 'highest_ratio', 'topic_ratio']
topictable['category'] = documents_merge['category']

topic1 ~ topic4 칼럼 추가

In [17]:
for index, simplex in enumerate(topictable.topic_ratio):
    for topic in simplex:
        if topic[0]==1:
            topictable.loc[index,'topic1'] = topic[1]
        elif topic[0]==2:
            topictable.loc[index,'topic2'] = topic[1]
        elif topic[0]==3:
            topictable.loc[index,'topic3'] = topic[1]
        elif topic[0]==4:
            topictable.loc[index,'topic4'] = topic[1]
        else:
            print('aaa')

In [18]:
topictable = topictable.fillna(0)

In [19]:
topictable = pd.merge(topictable, documents_merge, how='left')

In [20]:
topictable.drop('review', 1).to_csv('./LDA_results.csv', encoding='UTF-8')

In [21]:
topictable.loc[topictable.highest_topic==1].attractions

5     Gwangandaegyo_Bridge-Busan Nurimaru_APAC_House...
7     Jamsil_Baseball_Stadium-Seoul Lotte_Cinema_Lot...
9     Nami_Island-Chuncheon_Gangwon_do Chuncheon_Mye...
11    Coex_Aquarium-Seoul Starfield_COEX_Mall-Seoul ...
12    Yeouido_Hangang_Park-Seoul Yeouido_Park-Seoul ...
13                          Everland-Yongin_Gyeonggi_do
19                                        63_City-Seoul
20    Daegwanryeong_Samyang_Ranch-Pyeongchang_gun_Ga...
26    Phoenix_Snow_Park-Pyeongchang_gun_Gangwon_do Y...
27    Gapyeong_Rail_Park-Gapyeong_gun_Gyeonggi_do Ga...
31    Songdo_Central_Park-Incheon Songdo_Beach-Busan...
33        Seoul_World_Cup_Stadium-Seoul MBC_WORLD-Seoul
35    Incheon_Airport_Transit_Tour-Incheon AREX_Airp...
Name: attractions, dtype: object

In [22]:
topictable.loc[topictable.highest_topic==2].attractions

0     Myeongdong_Shopping_Street-Seoul Myeong_dong_C...
1     Dongdaemun_Market-Seoul Migliore_Dongdaemun_Sh...
3     Sinchon-Seoul Yonsei_University_Sinchon_Campus...
4     Itaewon-Seoul Itaewon_Land-Seoul Itaewon_Antiq...
6               Gangnam_Underground_Shopping_area-Seoul
15                       Gamcheon_Culture_Village-Busan
17                                     Garosu_gil-Seoul
22    Cheongdam_Fashion_Street-Seoul Hyundai_Departm...
24    BEXCO_Busan_Exhibition_Convention_Center-Busan...
34    Seomun_Market-Daegu Daegu_Metro-Daegu Daegu_Ar...
36    Jeonju_Hanok_Village-Jeonju_Jeollabuk_do Omokd...
Name: attractions, dtype: object

In [23]:
topictable.loc[topictable.highest_topic==3].attractions

2     Jogyesa_Temple-Seoul Jongmyo_Shrine-Seoul Jong...
16    Gwanghwamun_Square-Seoul Sejong_Center-Seoul K...
18    Bulguksa_Temple-Gyeongju_Gyeongsangbuk_do Bomu...
25    DMZ-Paju_Gyeonggi_do The_Third_Tunnel-Paju_Gye...
29    Wolmido-Incheon Wolmi_Park-Incheon Wolmi_Theme...
32    Hwaseong_Fortress-Suwon_Gyeonggi_do Hwaseong_H...
37    Donghwasa-Daegu Palgongsan_Cable_Car-Daegu Gat...
38    Yonin_Natural_Recreation_Forest-Yongin_Gyeongg...
Name: attractions, dtype: object

In [24]:
topictable.loc[topictable.highest_topic==4].attractions

8     Dongmun_Market-Jeju_Jeju_Island Jeju_Folklore_...
10           Jagalchi_Market-Busan Yongdusan_Park-Busan
14        Yonggungsa_Temple-Incheon Gijang_Market-Busan
21        Taejongdae-Busan Taejongdae_Resort_Park-Busan
23    Seoraksan_National_Park-Sokcho_Gangwon_do Ulsa...
28    Mireuksan_Mountain-Tongyeong_Gyeongsangnam_do ...
30    Windy_Hill-Geoje_Gyeongsangnam_do Geoje_Okpo_B...
Name: attractions, dtype: object

1번 (도시 - 조용한)
> nami, bike, aquarium, everland, beach, rid, haeunda, hangang, incheon, franc, yeouido, basebal, blossom, cherri, cycl, gapyeong, petit, coex, arex, songdo, safari, sonata, ferri, island, bicycl, roller, layov, cruis, firework, coaster
>
> 예시 : 광안대교, 잠실야구장, 롯데시네마, 나미섬, 코엑스아쿠아리움, 여의도한강공원, 63빌딩, 에버랜드

2번 (도시 - 번화가의)
> shop, street, cosmet, myeongdong, brand, cathedr, food, jeonju, product, cloth, church, fashion, store, skincar, skin, mask, bargain, cathol, namdaemun, accessori, hanok, gamcheon, shopper, makeup, itaewon, lobster, trendi, sampl, heaven, mass
>
> 예시 : 명동, 신촌, 홍대, 이태원, 강남, 가로수길, 청담패션거리, 현대백화점, ~~한옥마을~~

3번 (역사 - 조용한)
> stream, palac, templ, north, fortress, suwon, tunne, lantern, gyeongbokgung, gwanghwamun, gyeongju, shrine, hwaseong, cheonggyecheon, buddha, border, sejong, seoraksan, guard, bulguksa, infiltr, dora, ceremoni, dorasan, soldier, squar, wall, king, histori 
>
> 예시 : 광화문, 세종문화회관, 불국사, 월미도, 화성, 동화사, 제3터널, 팔공산

4번 (자연의 자연 - 조용한)
> jeju, seafood, hike, sunris, climb, trail, seongsan, crater, rock, hallasan, peak, diver, ilchulbong, summit, yeongsi, format, volcan, seongpanak, eorimok, mysteri, volcano, dragon, jagalchi, seogwipo, gwaneumsa, yongdusan, lava, dongmun, cliff, neutral
>
> 예시 : 제주도, 자갈치시장, 용두산, 용궁사, 태종대설악산, 울산바위, 미륵산, 거제도

# Mapping2 

Given (40 - 1) 범주로 묶은 관광지 리뷰로 학습한 LDA를 이용하여, {162개의 관광지}의 토픽을 뽑아보자,

리니어 모델 피팅할 때는 설문조사 데이터에서 y인 만족 방문지가 범주로 묶여있어서 162개를 40-1개로 뭉쳤는데

이미 모델(`demographics_analysis.Rmd`)은 만든 다음엔 X변수인 국적, 목적, 연령 변수만 들어오면 관심 토픽을 예측을 할 수 있기에

{162개의 관광지}의 토픽을 뽑아서 추천을 하도록 하자.

In [55]:
documents2 = documents.loc[documents.category!=0, :].reset_index(drop=True)
documents2.head()

,attr_index,attraction,review,category
0,0,Myeongdong_Shopping_Street-Seoul,"It's cosmetics, cosmetics and more cosmetics. ...",1
1,1,Myeong_dong_Cathedral-Seoul,A place of quite amongst the hustle and bustle...,1
2,2,Chuncheon_Myeongdong_Street-Chuncheon_Gangwon_do,BEAUTY PRODUCTS AREA. If you wanna shop for al...,1
3,3,Gwangandaegyo_Bridge-Busan,So prettyyyyyy!!. My friend fell in love at fi...,6
4,4,Nurimaru_APAC_House-Busan,Gorgeous historical attraction! Loved it. The ...,6


In [26]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MASTER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
stemmer = PorterStemmer()

In [28]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [29]:
processed_docs2 = documents2['review'].map(preprocess)
#processed_docs2

`dictionary`는 이미 위에서 학습된 것 사용

In [30]:
bow_corpus2 = [dictionary.doc2bow(doc) for doc in processed_docs2] # word frequency for each attraction
len(bow_corpus2)

162

마찬가지로, `ldamodel` 역시 이미 위에서 학습된 것 사용

In [64]:
topictable2 = make_topictable_per_doc(ldamodel, bow_corpus2, processed_docs2.index)
topictable2.columns = ['category', 'highest_topic', 'highest_ratio', 'topic_ratio']
topictable2['category'] = documents2['category']

In [65]:
documents2.category

0       1
1       1
2       1
3       6
4       6
       ..
157    26
158    30
159    30
160    12
161    34
Name: category, Length: 162, dtype: int32

In [66]:
for index, simplex in enumerate(topictable2.topic_ratio):
    for topic in simplex:
        if topic[0]==1:
            topictable2.loc[index,'topic1'] = topic[1]
        elif topic[0]==2:
            topictable2.loc[index,'topic2'] = topic[1]
        elif topic[0]==3:
            topictable2.loc[index,'topic3'] = topic[1]
        elif topic[0]==4:
            topictable2.loc[index,'topic4'] = topic[1]
        else:
            print('aaa')

In [68]:
topictable2 = topictable2.fillna(0)

In [78]:
topictable2 = pd.concat([topictable2, documents2], axis=1)

In [80]:
topictable2.drop('review', 1).to_csv('./LDA_results2.csv', encoding='UTF-8')